# Intro
As computer science students with a deep interest in data science, we, Hila Giladi (add id) and Kfir Shuster (add id), have chosen to focus on sleep duration.
Recognizing that quality sleep is crucial for everyone's daily functioning and overall health, we are committed to leveraging data science to help people worldwide achieve better sleep patterns and, consequently, improve their quality of life.

# The problem
In our modern world, where the boundaries between day and night blur and the demanding requirements of digital life continue to grow, we are witnessing an alarming sleep crisis that is intensifying. The data collected in our research presents a particularly troubling picture: an average of just 4.5 hours of sleep - a concerning and significant gap from the accepted medical recommendation of 7-9 hours of sleep per day. This chronic lack of sleep reflects the complex reality of contemporary society, where technology, work pressures, and the desire to accomplish everything push sleep to the bottom of our priority list. The factors contributing to this crisis are diverse and intertwined: from the blue screens illuminating our lives until late night hours, through demanding jobs requiring 24/7 availability, to the constant social pressure to stay connected and active. This phenomenon gains additional significance in the global era, where time zones blur and the expectation for immediate response has become the norm. The data at our disposal serves as a wake-up call - literally and figuratively - compelling us to confront the implications of the "always awake" culture we've developed, and to seek creative and sustainable solutions that will allow us to rebalance our natural sleep-wake cycle.


# The importance of the solution
Sleep is a fundamental aspect of human health and well-being, with particular significance for mental health.


# How we're going to do it:
This project aims to develop a predictive model for sleep duration using a comprehensive dataset that captures various physiological, behavioral, and environmental factors. The dataset includes continuous monitoring of multiple variables including heart rate, blood pressure, skin temperature, activity levels, and various lifestyle and mental health indicators.
By leveraging machine learning techniques to predict sleep duration, we aim to:
1. Identify key factors that influence sleep patterns
2. Understand the relationship between daily activities, stress levels, and sleep duration
3. Develop a model that can potentially help in early intervention for sleep-related issues
The dataset's unique combination of physiological measurements (heart rate, blood pressure, skin temperature), behavioral metrics (activity levels, social interaction), and mental health indicators (stress level, mental health status, resilience factors) provides a rich foundation for exploring the complex interplay between various life factors and sleep patterns.
Through this analysis, we hope to contribute to the broader understanding of sleep health and potentially develop tools that could help individuals and healthcare providers in monitoring and improving sleep patterns, particularly in the context of mental health management.

# Data Collection and Selection Process
Before beginning our research, we needed to find the right dataset for analyzing sleep patterns. We searched through many different datasets from health organizations and research groups, looking at how well each one could help us understand sleep patterns. Our goal was to find data that would tell us the most complete story about sleep and what affects it. After reviewing many options, we chose this dataset because it gives us an excellent mix of different types of information. It tracks physical measurements like heart rate and blood pressure, along with important details about people's daily lives - their work hours, stress levels, and how they're feeling. What made this dataset stand out was that it collects information every hour and includes both physical health measurements and mental well-being indicators. This means we can look at how sleep connects to many different parts of people's lives. With over 52,000 entries, the dataset is large enough to help us find reliable patterns and connections. We believe this rich combination of different types of information will help us better understand what affects people's sleep and how we might be able to improve it.

# Data Analysis
Let's transform our collected data into a structured pandas data frame to examine the information we've gathered. This will give us a clear view of our dataset's contents and help us understand what we're working with.

In [ ]:
import pandas as pd

In [ ]:
# read data_base
df = pd.read_csv('mental_health_monitoring_dataset.csv')
display(df)

# Cleaning the data
Let’s remove all the rows with nan values:

In [ ]:
original_len = len(df)
df = df.dropna()
print(f"number of removed rows: {original_len - len(df)}")

As we can see there are no rows with nan values.
Let's check for duplicates:

In [ ]:
df[df.duplicated()].shape[0]

As we can see there are no duplicates rows in the dataset.

Let’s see how many different values there are in our prediction column, Sleep_Duration:

In [ ]:
df['Sleep_Duration'].unique()

The current output is an array with all the different values in the column Sleep_Duration.

Delete rows with outliers values in the prediction column, Sleep Duration < 1

In [ ]:
original_len = len(df)
df = df[df['Sleep_Duration'] >= 1]
print(f"number of removed rows: {original_len - len(df)}")
display(df)

Delete irrelevant columns:
'Location_Longitude', 'Location_Latitude', 'Fuel_Consumption', 'Timestamp', 'Galvanic_Skin_Response'
We decided to delete those columns because they have no effect on sleep duration.

In [ ]:
df = df.drop(['Location_Longitude', 'Location_Latitude', 'Fuel_Consumption', 'Timestamp', 'Galvanic_Skin_Response'], axis=1)
display(df)

Delete rows with outlier values, skin temperatures below 35 are unusual.
Let's remove those rows.

In [ ]:
original_len = len(df)
df = df[df['Skin_Temperature'] >= 35]
print(f"number of removed rows: {original_len - len(df)}")
display(df)

Remove records where the values in the columns Mood and Stress_Level are logically incompatible
   1. Mood is 'High' and Stress_Level is either 'Anxious', 'Sad', or 'Irritable'
   2. Mood is 'Low' and Stress_Level is 'Happy'

In [ ]:
original_len = len(df)
df = df[
    ~((df['Mood'] == 'High') & df['Stress_Level'].isin(['Anxious', 'Sad', 'Irritable'])) &
    ~((df['Mood'] == 'Low') & (df['Stress_Level'] == 'Happy'))
]
print(f"number of removed rows: {original_len - len(df)}")

Change text values to numeric values in the columns Driving_Conditions, Stress_Level, Mental_Health_History, Mental_Health_Status, Mood

In [ ]:
from sklearn.preprocessing import LabelEncoder
columns_to_encode = ['Driving_Conditions', 'Stress_Level', 'Mental_Health_History', 'Mental_Health_Status', 'Mood']
# for transform new data later with same encodings or reverse the numeric values back to text
encoders = {}
mapping_text_to_numeric = {}

for col in columns_to_encode:
    encoders[col] = LabelEncoder()
    df[col] = encoders[col].fit_transform(df[col])
    # mapping dictionary
    mapping_text_to_numeric = dict(zip(encoders[col].classes_, encoders[col].transform(encoders[col].classes_)))

print(df)

Categorize numeric values in the column Activity_Levels

In [ ]:
import numpy as np

df['Activity_Levels'] = np.where(df['Activity_Levels'] <= 3000, 1, np.where(df['Activity_Levels'] <= 10000, 2, 3))
display(df)

Normalize the numeric values in the columns Heart_Rate, Blood_Pressure_systolic, Blood_Pressure_diastolic

In [ ]:
from sklearn.preprocessing import MinMaxScaler

columns_to_normalize = ['Heart_Rate', 'Blood_Pressure_Systolic', 'Blood_Pressure_Diastolic']
scaler = MinMaxScaler()
df[columns_to_normalize] = scaler.fit_transform(df[columns_to_normalize])

display(df)